In [1]:
import QuantLib as ql
import numpy as np
import pandas as pd
from datetime import datetime
from model_settings import ms
from itertools import product


pricing settings:
Actual/365 (Fixed) day counter
New York stock exchange calendar
compounding: continuous
frequency: annual



In [2]:
calculation_datetime = datetime.today()
calculation_date = ql.Date(
    calculation_datetime.day,
    calculation_datetime.month,
    calculation_datetime.year
)
r = 0.05
g = 0.02
s = 100.00
k = 50
w = 'put'
t = 360

v0 = 0.01 
kappa = 0.2
theta = 0.02
rho = -0.75
eta = 0.5

n_paths = 100000

past_fixings = 0 # Empty because this is a new contract

averaging_frequency = 5

In [3]:
my_geometric = ms.ql_asian_price(
            s,k,t,r,g,calculation_datetime, w,
            'geometric',past_fixings,averaging_frequency,
            kappa,theta,rho,eta,v0
            )

my_arithmetic = ms.ql_asian_price(
            s,k,t,r,g,calculation_datetime, w,
            'arithmetic',past_fixings,averaging_frequency,
            kappa,theta,rho,eta,v0
            )

In [4]:
vanilla = ms.ql_heston_price(
    s,k,t,r,g,w, 
    kappa, theta, rho, eta, v0, 
    calculation_datetime
)
print(f"\nvanilla: {vanilla}\n\ngeometric: {my_geometric}\n\narithmetic: {my_arithmetic}\n")


vanilla: 44.737210108887545

geometric: 46.241440446606575

arithmetic: 46.125983965125606



In [ ]:
# K = np.arange(int(s*0.5),int(s)*1.5+1,5).astype(int).tolist()
K = np.arange(s*0.5,s*1.5,5).astype(int).tolist()
averaging_frequencies = [5,10,15,30,45]
T = [90,180,360]
past_fixings = [0]

features = pd.DataFrame(
    product(
        [s],K,T,
        averaging_frequencies,
        ['call','put'],
        ['geometric','arithmetic'],
        past_fixings,
        [theta],
        [kappa],
        [rho],
        [eta],
        [v0],
        [r],
        [g],
        [calculation_datetime]
    ),
    columns = [
        'spot_price',
        'strike_price',
        'days_to_maturity',
        'averaging_frequency',
        'w',
        'averaging_type',
        'past_fixings',
        'kappa','theta','rho','eta','v0',
        'risk_free_rate',
        'dividend_rate',
        'calculation_date',
    ]
)

features['vanilla_price'] = ms.vector_heston_price(features)
features['asian_price'] = ms.vector_asian_price(features)
pd.set_option("display.float_format", '{:.2f}'.format)
pd.set_option("display.max_rows",None)
features

In [ ]:
pd.reset_option("display.max_rows")